## A molecular single-cell lung atlas of lethal COVID-19
##### referene: https://www.nature.com/articles/s41586-021-03569-1
##### platform: Illumina NovaSeq 6000 and snRNA-seq using a droplet-based platform (10x Genomics)
##### overall design: Single-nuclei RNA sequencing of 116,314 cells from 20 frozen lungs obtained from 19 COVID-19 decedents and seven control patients.
##### control: 7 patients vs disease: 19 covid-19 patients

In [ ]:
import scanpy as sc
import scvi
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

import seaborn as sns

In [ ]:
## read data
data_path = '/home/bonny/Documents/bioinformatics_workspace/datasets/single_cell_pipline/covid-19/GSM5226574_C51ctr_raw_counts.csv.gz'

adata = sc.read_csv(data_path).T
adata

In [ ]:
adata.X.shape

# 1) Doublet removal 

In [ ]:
adata

In [ ]:
## keep genes that are captured in at least 10 cells
sc.pp.filter_genes(adata, min_cells=10)
adata

In [ ]:
## keep only 2000 top variable genes
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True, flavor='seurat_v3')
adata

In [ ]:
adata.var

In [ ]:
## train SCVI model on adata
scvi.model.SCVI.setup_anndata(adata)
vae = scvi.model.SCVI(adata)
vae.train()

In [ ]:
## train Solo model (external)
solo = scvi.external.SOLO.from_scvi_model(vae)
solo.train()

In [ ]:
df_solo_predict = solo.predict()
df_solo_predict['prediction'] = solo.predict(soft=False)
df_solo_predict['diff'] = df_solo_predict.doublet - df_solo_predict.singlet
df_solo_predict

In [ ]:
df_solo_predict.groupby('prediction').count()

In [ ]:
sns.displot(df_solo_predict[df_solo_predict.prediction == 'doublet'], x = 'diff')

In [ ]:
doublets = df_solo_predict[(df_solo_predict['prediction'] == 'doublet') & (df_solo_predict['diff'] > 1) ]
doublets

In [ ]:
adata

# 2) Preprocessing data

In [ ]:
adata = sc.read_csv(data_path).T
adata

In [ ]:
adata.obs['doublet'] = adata.obs.index.isin(doublets.index)

In [ ]:
adata.obs

In [ ]:
adata = adata[~adata.obs.doublet]
adata

## lable mitochondial genes

In [ ]:
adata.var['mt'] = adata.var.index.str.startswith('MT-')
adata.var

## lable ribosomal genes

In [ ]:
import pandas as pd

In [ ]:
broad_url = 'https://www.gsea-msigdb.org/gsea/msigdb/human/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt'

In [ ]:
ribosomal_genes = pd.read_table(broad_url, skiprows=2, header=None)
ribosomal_genes

In [ ]:
adata.var['ribosomal'] = adata.var_names.isin(ribosomal_genes[0].values)
adata.var

In [ ]:
## calculate QC metrix on 'mt' and 'robosomal' genes

sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribosomal'], percent_top=None, log1p=False, inplace=True)
adata.obs

In [ ]:
adata

In [ ]:
adata.var

In [ ]:
adata.var.sort_values('n_cells_by_counts')

In [ ]:
sc.pp.filter_genes(adata, min_cells=3)
adata.var.sort_values('n_cells_by_counts')

In [ ]:
adata.obs.sort_values('total_counts')

In [ ]:
# sc.pp.filter_cells(min_genes=200)

In [ ]:
sc.pl.violin(adata, ['total_counts', 'n_genes_by_counts', 'pct_counts_mt', 'pct_counts_ribosomal'], jitter=0.4, multi_panel=True)

In [ ]:
adata

In [ ]:
import numpy as np

In [ ]:
## filter data on cells with 98 percentile of gene counts
upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, 0.98)
upper_lim

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < upper_lim]

In [ ]:
adata.obs

In [ ]:
## filter data on mitochondiral
adata = adata[adata.obs.pct_counts_mt < 20]

In [ ]:
## filter data on ribosomal genes
adata = adata[adata.obs.pct_counts_ribosomal < 2]

In [ ]:
adata

# Nomalization

In [ ]:
adata.X.sum(axis=1)

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4) ## normalize each cell by 10,000 UMI

In [ ]:
adata.X.sum(axis=1)

In [ ]:
sc.pp.log1p(adata) # apply log to counts

In [ ]:
adata.X.sum(axis=1)

# Freez filter and normalized data

In [ ]:
# Freeze filter and normalized data
adata.raw = adata 

# Clustering

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
adata.var

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata = adata[:, adata.var.highly_variable]

In [ ]:
adata

In [ ]:
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt', 'pct_counts_ribosomal'])

In [ ]:
adata.var

In [ ]:
sc.pp.scale(adata, max_value=10) # scale data to unit variance and zero mean. And clip at max value 10.

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

In [ ]:
sc.pp.neighbors(adata, n_pcs=30)

In [ ]:
adata

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution=0.5)

In [ ]:
adata.obs

In [ ]:
sc.pl.umap(adata, color=['leiden'])

# Integrate all the samples

In [ ]:
data_dir = '/home/bonny/Documents/bioinformatics_workspace/datasets/single_cell_pipline/covid-19/'

In [ ]:
import os

In [ ]:
## Function to pre-process all the samples

def sample_preprocessing(input_csv: str, sample: str):

    ## 1) find doubets 

    ## load .csv file
    adata = sc.read_csv(input_csv).T

    ## filter genes : at least in 10 cells
    sc.pp.filter_genes(adata, min_cells=10)

    ## get highly variable 2000 genes 
    sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True, flavor='seurat_v3')

    ## load and train SCVI model to identify 'doublets'
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()

    ## Train solo model to predict "doublets". It needs train scvi model to be passed.

    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()

    df = solo.predict()
    df['prediction'] = solo.predict(soft=False)
    df['diff'] = df.doublet - df.singlet
    doublets = df[(df['prediction']=='doublet') & (df['diff']>1 )]


    ## 2) pre-processing

    ## assign samples
    adata = sc.read_csv(input_csv).T
    adata.obs['sample'] = sample

    ## assign doublets
    adata.obs['doublet'] = adata.obs.index.isin( doublets.index )
    adata = adata[~ adata.obs.doublet]

    ## label 'mitochondrial' and 'ribosomal' genes
    sc.pp.filter_cells(adata, min_genes=200) ## filter cells with fewer than 200 genes
    adata.var['mt'] = adata.var_names.str.startswith( 'MT-' ) ## label mitochondrial genes
    adata.var['ribosomal'] = adata.var_names.isin( ribosomal_genes[0].values ) ## label ribosomal genes

    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribosomal'], percent_top=None, log1p=False, inplace=True)
    upper_limit = np.quantile(adata.obs.n_genes_by_counts.values, 0.98)
    adata = adata[adata.obs.n_genes_by_counts < upper_limit]
    adata = adata[adata.obs.pct_counts_mt < 20]
    adata = adata[adata.obs.pct_counts_ribosomal < 2]

    return adata



In [ ]:
## create sample_name : sample_path dictionary
sample_dict = dict()
for fi in sorted( os.listdir(data_dir) ):
    file_extension = os.path.splitext(fi)[-1]
    if file_extension == '.gz':
        file_path = data_dir + fi
        sample = fi.split('_')[1]
        sample_dict[sample] = file_path

In [ ]:
sample_dict

In [ ]:
sample_dict.keys()

In [ ]:
selected_samples = ['C51ctr', 'C52ctr', 'L01cov', 'L03cov']

In [ ]:
output = []
for sample, sample_csv in sample_dict.items():
    # if sample in selected_samples:
    print(sample, sample_csv)
    print()
    adata = sample_preprocessing(sample_csv, sample)
    adata.write_h5ad(data_dir + f'{sample}.h5ad')
    # output.append(sample_preprocessing(sample_csv, sample))
    print('='*100)
    print()

In [ ]:
len(output)

# Integration 

In [ ]:
adata = sc.concat(output)
adata

In [ ]:
adata.obs

In [ ]:
sc.pp.filter_genes(adata, min_cells=10)

In [ ]:
adata.X = csr_matrix(adata.X)
adata.X

In [ ]:
adata.write_h5ad(data_dir + 'samples_combibed.h5ad')

In [ ]:
bdata = sc.read_h5ad(data_dir + 'samples_combibed.h5ad')

In [ ]:
bdata

In [ ]:
## save the data as counts
bdata.layers['counts'] = bdata.X.copy()

In [ ]:
## Normalization of counts
sc.pp.normalize_total(bdata, target_sum=1e4) # normaliza to sum 10000
sc.pp.log1p(bdata) ## log normalize the data 
bdata.raw = bdata ## save log normaliza data to raw object


In [ ]:
bdata.obs.head(3)

In [ ]:
## reduce the gene to highly variable 3000 genes (import to run scVI)
sc.pp.highly_variable_genes(bdata, n_top_genes = 3000, subset=True, layer='counts', flavor='seurat_v3', batch_key='sample') 

In [ ]:
bdata

In [ ]:
## Set ip scVI model

scvi.model.SCVI.setup_anndata(bdata, 
                              layer='counts',
                              categorical_covariate_keys=['sample'],
                              continuous_covariate_keys=['total_counts', 'pct_counts_mt', 'pct_counts_ribosomal']
                             )

In [ ]:
model = scvi.model.SCVI(bdata)

In [ ]:
model.train()

In [ ]:
bdata

In [ ]:
bdata.obsm['X_scVI'] = model.get_latent_representation()

In [ ]:
bdata.layers['scvi_normalized'] = model.get_normalized_expression(library_size=1e4)

In [ ]:
sc.pp.neighbors(bdata, use_rep = 'X_scVI')

In [ ]:
sc.tl.umap(bdata)
sc.tl.leiden(bdata, resolution=0.5)

In [ ]:
sc.pl.umap(bdata, color = ['leiden', 'sample'], frameon=False)

In [ ]:
bdata.write_h5ad(data_dir + 'integrated.h5ad')

# Find markers / Label cell types

In [ ]:
sc.tl.rank_genes_groups(bdata, 'leiden')

In [ ]:
# sc.pl.rank_genes_groups(bdata, n_genes=15, sharey=False)

In [ ]:
markers = sc.get.rank_genes_groups_df(bdata, None)
markers = markers[ (markers.pvals_adj < 0.05) & (markers.logfoldchanges > 0.5) ]

In [ ]:
markers

In [ ]:
markers_scvi = model.differential_expression(groupby='leiden')

In [ ]:
markers_scvi

In [ ]:
markers_scvi = markers_scvi[ (markers_scvi['is_de_fdr_0.05']) & (markers_scvi.lfc_mean > 0.5) ]
markers_scvi

In [ ]:
cell_type = {"0":"Macrophage",
"1":"Fibroblast",
"2":"CD4+ T-cell",
"3":"AT2",
"4":"AT1",
"5":"CD8+ T-cell",
"6":"Endothelial cell",
"7":"Plasma cell",
"8":"Monocyte",
"9":"B-cell",
"10":"Aerocyte",
"11":"Dendritic cell",
"12":"Neuronal cell",
"13":"Pericyte",
"14":"Smooth muscle cell",
"15":"AT2"}


In [ ]:
sc.pl.umap(bdata, color=['leiden'], frameon=False, legend_loc='on data')

In [ ]:
sc.pl.umap(bdata, color=['PTPRC','MRC1', 'ITGAX'], frameon=False, layer="scvi_normalized")

In [ ]:
bdata.obs

In [ ]:
bdata.obs['cell_type'] = bdata.obs.leiden.map(cell_type)

In [ ]:
bdata.obs

In [ ]:
sc.pl.umap(bdata, color=['cell_type'], frameon=False, legend_loc="on data")

In [ ]:
bdata

In [ ]:
bdata.uns['scvi_markers'] = markers_scvi
bdata.uns['markers'] = markers

In [ ]:
bdata.write(data_dir +  'integrated.h5ad')

In [ ]:
model.save(data_dir + 'model.model')

# Analysis

### Counting Cells

In [ ]:
cdata = sc.read_h5ad(data_dir + 'integrated.h5ad')

In [ ]:
cdata.obs.sample.unique().tolist()